In [1]:
import numpy as np
import pandas as pd
import os
import requests
import bs4
import json
import seaborn as sns
import re
from sklearn.compose import ColumnTransformer

In [2]:
url = 'https://lol.fandom.com/wiki/Leaguepedia:News/{0}/{1}'

In [3]:
jan_2021 = url.format(2021, 'January')
response = requests.get(jan_2021)

In [4]:
soup = bs4.BeautifulSoup(response.content)

In [5]:
rows = soup.find_all('tr')

In [6]:
rows[22].text.strip('edit')

'January 31'

In [7]:
region = rows[23].find('td').text
team_name = rows[23].find_all('td')[1].find('span', attrs={'class': 'teamname'}).text
news = rows[23].find_all('td')[2].text

In [8]:
news

'El Couch (Head Coach) joins. CougarDai and Nicaber join as substitutes.'

In [9]:
re.split('[ ]*join[s\., ]*', news)

['El Couch (Head Coach)', 'CougarDai and Nicaber', 'as substitutes.']

In [10]:
path = os.path.join('data', '2020_matches.csv')
matches = pd.read_csv(path)
matches['team'] = matches['team'].replace('DAMWON Gaming', 'DWG KIA')

/Users/flynnosullivan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
matches['league'].value_counts()
lol = matches[matches['league'].isin (['LPL', 'LCK', 'LCS', 'LEC'])]

In [63]:
only_more_than_10 = (lol['player'].value_counts() >= 5)
exp_players = only_more_than_10[only_more_than_10].index.to_list()
only_exp = lol[lol['player'].isin(exp_players)]

In [67]:
def helper_team(row):
    name = row['player']
    teams = matches[matches['player'] == name]['team'].value_counts().index.to_list()
    return teams

def helper_pos(row):
    name = row['player']
    position = matches[matches['player'] == name]['position'].value_counts().idxmax()
    return position

def helper_games_played(row):
    name = row['player']
    num_matches = len(lol[lol['player'] == name])
    return num_matches

df = pd.DataFrame(data={'player': only_exp['player'].value_counts().index}, index=only_exp['player'].value_counts().index)
df['team'] = df.apply(helper_team, axis=1)
df['position'] = df.apply(helper_pos, axis=1)
df['games_played'] = df.apply(helper_games_played, axis=1)

In [167]:
df['win_rate'] = only_exp.groupby('player')['result'].mean()
df['avg_kills'] = only_exp.groupby('player')['kills'].mean()
df['avg_deaths'] = only_exp.groupby('player')['deaths'].mean()
df['avg_assists'] = only_exp.groupby('player')['assists'].mean()
df['avg_teamkills'] = only_exp.groupby('player')['teamkills'].mean()
df['avg_teamdeaths'] = only_exp.groupby('player')['teamdeaths'].mean()
df['avg_doublekills'] = only_exp.groupby('player')['doublekills'].mean()
df['avg_triplekills'] = only_exp.groupby('player')['triplekills'].mean()
df['avg_quadrakills'] = only_exp.groupby('player')['quadrakills'].mean()
df['avg_pentakills'] = only_exp.groupby('player')['pentakills'].mean()
df['firstblood_rate'] = only_exp.groupby('player')['firstblood'].mean()
df['firstbloodvictim_rate'] = only_exp.groupby('player')['firstbloodvictim'].mean()
df['avg_team_kpm'] = only_exp.groupby('player')['team kpm'].mean()
df['avg_ckpm'] = only_exp.groupby('player')['ckpm'].mean()
df['avg_dmg_to_champions'] = only_exp.groupby('player')['damagetochampions'].mean()
df['avg_dpm'] = only_exp.groupby('player')['dpm'].mean()
df['avg_dmgshare'] = only_exp.groupby('player')['damageshare'].mean()
df['avg_dmgmitigatedperminute'] = only_exp.groupby('player')['damagemitigatedperminute'].mean()
df['avg_wardsplaced'] = only_exp.groupby('player')['wardsplaced'].mean()
df['avg_wpm'] = only_exp.groupby('player')['wpm'].mean()
df['avg_wardskilled'] = only_exp.groupby('player')['wardskilled'].mean()
df['avg_controlwardsbought'] = only_exp.groupby('player')['controlwardsbought'].mean()
df['avg_visionscore'] = only_exp.groupby('player')['visionscore'].mean()
df['avg_totalgold'] = only_exp.groupby('player')['totalgold'].mean()
df['avg_earnedgold'] = only_exp.groupby('player')['earnedgold'].mean()
df['avg_earngedgpm'] = only_exp.groupby('player')['earned gpm'].mean()
df['avg_earnedgoldshare'] = only_exp.groupby('player')['earnedgoldshare'].mean()
df['avg_goldspent'] = only_exp.groupby('player')['goldspent'].mean()
df['avg_cs'] = only_exp.groupby('player')['total cs'].mean()
df['avg_minionkills'] = only_exp.groupby('player')['minionkills'].mean()
df['avg_monsterkillsownjg'] = only_exp.groupby('player')['monsterkillsownjungle'].mean()
df['avg_monsterkillsenemyjg'] = only_exp.groupby('player')['monsterkillsenemyjungle'].mean()
df['avg_cspm'] = only_exp.groupby('player')['cspm'].mean()
df['avg_goldat10'] = only_exp.groupby('player')['goldat10'].mean()
df['avg_xpat10'] = only_exp.groupby('player')['xpat10'].mean()
df['avg_csat10'] = only_exp.groupby('player')['csat10'].mean()
df['avg_golddiffat10'] = only_exp.groupby('player')['golddiffat10'].mean()
df['avg_xpdiffat10'] = only_exp.groupby('player')['xpdiffat10'].mean()
df['avg_csdiffat10'] = only_exp.groupby('player')['csdiffat10'].mean()
df['avg_killsat10'] = only_exp.groupby('player')['killsat10'].mean()
df['avg_assistsat10'] = only_exp.groupby('player')['assistsat10'].mean()
df['avg_deathsat10'] = only_exp.groupby('player')['deathsat10'].mean()
df['avg_goldat15'] = only_exp.groupby('player')['goldat15'].mean()
df['avg_xpat15'] = only_exp.groupby('player')['xpat15'].mean()
df['avg_csat15'] = only_exp.groupby('player')['csat15'].mean()
df['avg_golddiffat15'] = only_exp.groupby('player')['golddiffat15'].mean()
df['avg_xpdiffat15'] = only_exp.groupby('player')['xpdiffat15'].mean()
df['avg_csdiffat15'] = only_exp.groupby('player')['csdiffat15'].mean()
df['avg_killsat15'] = only_exp.groupby('player')['killsat15'].mean()
df['avg_assistsat15'] = only_exp.groupby('player')['assistsat15'].mean()
df['avg_deathsat15'] = only_exp.groupby('player')['deathsat15'].mean()

In [210]:
mids = df[df['position'] == 'mid']
jgs = df[df['position'] == 'jng']
tops = df[df['position'] == 'top']
adcs = df[df['position'] == 'bot']
sups = df[df['position'] == 'sup']

In [113]:
df.columns.to_list()

['player',
 'team',
 'position',
 'games_played',
 'win_rate',
 'avg_kills',
 'avg_deaths',
 'avg_assists',
 'avg_teamkills',
 'avg_teamdeaths',
 'num_doublekills',
 'num_triplekills',
 'num_quadrakills',
 'num_pentakills',
 'firstblood_rate',
 'firstbloodvictim_rate',
 'avg_team_kpm',
 'avg_ckpm',
 'avg_dmg_to_champions',
 'avg_dpm',
 'avg_dmgshare',
 'avg_dmgmitigatedperminute',
 'avg_wardsplaced',
 'avg_wpm',
 'avg_wardskilled',
 'avg_controlwardsbought',
 'avg_visionscore',
 'avg_totalgold',
 'avg_earnedgold',
 'avg_earngedgpm',
 'avg_earnedgoldshare',
 'avg_goldspent',
 'avg_cs',
 'avg_minionkills',
 'avg_monsterkillsownjg',
 'avg_monsterkillsenemyjg',
 'avg_cspm',
 'avg_goldat10',
 'avg_xpat10',
 'avg_csat10',
 'avg_golddiffat10',
 'avg_xpdiffat10',
 'avg_csdiffat10',
 'avg_killsat10',
 'avg_assistsat10',
 'avg_deathsat10',
 'avg_goldat15',
 'avg_xpat15',
 'avg_csat15',
 'avg_golddiffat15',
 'avg_xpdiffat15',
 'avg_csdiffat15',
 'avg_killsat15',
 'avg_assistsat15',
 'avg_deathsat